In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random as r
from matplotlib import style
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QLabel, QVBoxLayout, QWidget, QApplication

class Grid:  # Grid

    # Initialization function
    def __init__(self, n, blocked_tiles):
        self.n = n
        self.blocked_tiles = blocked_tiles
        self.grid = np.zeros((n, n))

        for tile in blocked_tiles:
            self.grid[tile] = -1

    def isValid(self, pos):
        x, y = pos
        return 0 <= x < self.n and 0 <= y < self.n and self.grid[x, y] != -1  # Tile is not blocked

    # Get possible actions from a position
    def get_possible_acts(self, pos):
        x, y = pos
        acts = []
        for act, (dx, dy) in {'u': (-1, 0), 'd': (1, 0), 'l': (0, -1), 'r': (0, 1)}.items():
            new_pos = (x + dx, y + dy)
            if self.isValid(new_pos):
                acts.append((act, new_pos))
        return acts

    # Reset the grid
    def reset(self):
        self.grid = np.zeros((self.n, self.n))  # Clear the grid
        for tile in self.blocked_tiles:
            self.grid[tile] = -1

# Agent
import heapq
import numpy as np
import random as r

class Player:
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.2, planning_steps=5, kappa=0.2):
        self.env = env
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration vs exploitation
        self.planning_steps = planning_steps  # Number of planning steps for Dyna-Q
        self.kappa = kappa
        self.Q_steps_per_ep = []
        self.dynaQ_steps_per_ep = []
        self.dynaQPlus_steps_per_ep = []
        self.PrioritizedSweeping_steps_per_ep = []
        self.Qtable = {}
        self.model = {}
        self.time_since_visited = {}
        self.priority_queue = []
        for x in range(self.env.n):
            for y in range(self.env.n):
                if self.env.grid[x, y] != -1:  # Not blocked
                    self.Qtable[(x, y)] = {act: 0 for act, _ in self.env.get_possible_acts((x, y))}
                    self.model[(x, y)] = {act: None for act, _ in self.env.get_possible_acts((x, y))}
                    self.time_since_visited[(x, y)] = {act: 0 for act, _ in self.env.get_possible_acts((x, y))}

    def getAct(self, state):
        if r.uniform(0, 1) < self.epsilon:
            return r.choice(list(self.Qtable[state].keys()))  # Exploration
        else:
            return max(self.Qtable[state], key=self.Qtable[state].get)  # Exploitation

    def incQ(self, state, action, reward, next_state):
        if next_state not in self.Qtable:
            return
        best_next_action = max(self.Qtable[next_state], key=self.Qtable[next_state].get)  # maxQ'(s', a')
        td_target = reward + self.gamma * self.Qtable[next_state][best_next_action]  # R(s,a) + gamma * Q(s, a)
        td_error = td_target - self.Qtable[state][action]  # R(s,a) + gamma * Q(s, a) - Q(s, a)
        self.Qtable[state][action] += self.alpha * td_error  # Q(s, a) = Q(s, a) + alpha * { R(s,a) + [gamma * Q(s, a)] - Q(s, a) }
        return abs(td_error)

    def train_Q(self, episodes=2000):
        global q_learning_episodes, min_Path_Q
        for ep in range(episodes):
            state = (r.randint(0, self.env.n - 1), r.randint(0, self.env.n - 1))
            while not self.env.isValid(state) or state == (x_T, y_T):
                state = (r.randint(0, self.env.n - 1), r.randint(0, self.env.n - 1))

            while state != (x_T, y_T):
                action = self.getAct(state)
                next_state = [new_state for act, new_state in self.env.get_possible_acts(state) if act == action][0]
                reward = 1 if next_state == (x_T, y_T) else -(1/self.env.n)
                self.incQ(state, action, reward, next_state)
                state = next_state

            # Check if the path is optimal
            if ep == 1:
                min_Path_Q = 1000
            if ep > 2:
                path = self.path(start)
                if self.is_optimal_path(path) and ep > 2:
                    if len(path) < min_Path_Q:
                        q_learning_episodes = ep + 1
                        min_Path_Q = len(path)
            self.Q_steps_per_ep.append(len(self.path(start)) - 1)

    def train_DynaQ(self, episodes=2000):
        global dyna_q_episodes, min_Path_dynaQ
        for ep in range(episodes):
            while True:
                state = (r.randint(0, self.env.n - 1), r.randint(0, self.env.n - 1))
                if self.env.isValid(state) and state != (x_T, y_T):
                    break

            while state != (x_T, y_T):
                action = self.getAct(state)
                next_state = [new_state for act, new_state in self.env.get_possible_acts(state) if act == action][0]
                reward = 1 if next_state == (x_T, y_T) else -(1/self.env.n)
                self.incQ(state, action, reward, next_state)

                # Update the model
                self.model[state][action] = (next_state, reward)

                # Planning steps
                for _ in range(self.planning_steps):
                    sim_state = r.choice(list(self.model.keys()))
                    sim_action = r.choice(list(self.model[sim_state].keys()))
                    sim_next_state, sim_reward = self.model[sim_state][sim_action] if self.model[sim_state][sim_action] else (sim_state, 0)
                    self.incQ(sim_state, sim_action, sim_reward, sim_next_state)

                state = next_state

            # Check if the path is optimal
            if ep == 1:
                min_Path_dynaQ = 1000
            if ep > 2:
                path = self.path(start)
                if self.is_optimal_path(path):
                    if len(path) < min_Path_dynaQ:
                        dyna_q_episodes = ep + 1
                        min_Path_dynaQ = len(path)
            self.dynaQ_steps_per_ep.append(len(self.path(start)) - 1)

    def train_DynaQPlus(self, episodes=2000):
        global dyna_q_plus_episodes, min_Path_dynaQPlus
        for ep in range(episodes):
            while True:
                state = (r.randint(0, self.env.n - 1), r.randint(0, self.env.n - 1))
                if self.env.isValid(state) and state != (x_T, y_T):
                    break

            while state != (x_T, y_T):
                action = self.getAct(state)
                next_state = [new_state for act, new_state in self.env.get_possible_acts(state) if act == action][0]
                reward = 1 if next_state == (x_T, y_T) else -(1/self.env.n)
                self.incQ(state, action, reward, next_state)

                # Update the model and time since visited
                self.model[state][action] = (next_state, reward)
                self.time_since_visited[state][action] = ep

                # Planning steps with bonus reward
                for _ in range(self.planning_steps):
                    # Choose a state-action pair based on the bonus
                    max_bonus = -float('inf')
                    chosen_state = None
                    chosen_action = None

                    for s in self.Qtable:
                        for a in self.Qtable[s]:
                            bonus = self.kappa * np.sqrt(ep - self.time_since_visited[s][a])
                            if self.Qtable[s][a] + bonus > max_bonus:
                                max_bonus = self.Qtable[s][a] + bonus
                                chosen_state = s
                                chosen_action = a

                    sim_next_state, sim_reward = self.model[chosen_state][chosen_action] if self.model[chosen_state][chosen_action] else (chosen_state, 0)
                    self.incQ(chosen_state, chosen_action, sim_reward, sim_next_state)

                state = next_state

            # Check if the path is optimal
            if ep == 1:
                min_Path_dynaQPlus = 1000
            if ep > 2:
                path = self.path(start)
                if self.is_optimal_path(path):
                    if len(path) < min_Path_dynaQPlus:
                        dyna_q_plus_episodes = ep + 1
                        min_Path_dynaQPlus = len(path)
            self.dynaQPlus_steps_per_ep.append(len(self.path(start)) - 1)




    def prioritize(self, state, action, priority):
        heapq.heappush(self.priority_queue, (-priority, state, action))  # Push the priority with a negative sign to make it a max-heap

    def update_priorities(self, state, action, td_error):
        if td_error > self.kappa:  # Check if the td_error exceeds the threshold
            self.prioritize(state, action, td_error)  # Add to priority queue if it does

    def get_predecessors(self, state):
        predecessors = []
        for s in self.model:
            for a in self.model[s]:
                if self.model[s][a] is not None:  # Ensure the model entry is not None
                    ns, _ = self.model[s][a]
                    if ns == state:
                        predecessors.append((s, a))
        return predecessors


    def train_PrioritizedSweeping(self, episodes=2000):
        global prioritized_sweeping_episodes, min_Path_PrioritizedSweeping
        for ep in range(episodes):
            while True:
                state = (r.randint(0, self.env.n - 1), r.randint(0, self.env.n - 1))
                if self.env.isValid(state) and state != (x_T, y_T):
                    break

            while state != (x_T, y_T):
                action = self.getAct(state)
                next_state = [new_state for act, new_state in self.env.get_possible_acts(state) if act == action][0]
                reward = 1 if next_state == (x_T, y_T) else -(1/self.env.n)
                td_error = self.incQ(state, action, reward, next_state)

                # Update the model
                self.model[state][action] = (next_state, reward)
                self.update_priorities(state, action, td_error)

                # Planning steps with prioritized sweeping
                for _ in range(self.planning_steps):
                    if not self.priority_queue:
                        break
                    priority, sim_state, sim_action = heapq.heappop(self.priority_queue)
                    priority = -priority  # Convert back to positive priority
                    sim_next_state, sim_reward = self.model[sim_state][sim_action] if self.model[sim_state][sim_action] else (sim_state, 0)
                    td_error = self.incQ(sim_state, sim_action, sim_reward, sim_next_state)
                    self.update_priorities(sim_state, sim_action, td_error)

                    for pred_state, pred_action in self.get_predecessors(sim_state):
                        pred_td_error = self.incQ(pred_state, pred_action, sim_reward, sim_state)
                        self.update_priorities(pred_state, pred_action, pred_td_error)

                state = next_state

            # Check if the path is optimal
            if ep == 1:
                min_Path_PrioritizedSweeping = 1000
            if ep > 2:
                path = self.path(start)
                if self.is_optimal_path(path):
                    if len(path) < min_Path_PrioritizedSweeping:
                        prioritized_sweeping_episodes = ep + 1
                        min_Path_PrioritizedSweeping = len(path)
            self.PrioritizedSweeping_steps_per_ep.append(len(self.path(start)) - 1)    




    





    def path(self, start):
        if start not in self.Qtable:
            return []
        path = [start]
        state = start
        while state != (x_T, y_T):
            if len(path) > self.env.n * self.env.n:
                return path
            if not self.Qtable[state]:  # If current state has no available actions in Q-table, returns the path found so far.
                return path
            action = max(self.Qtable[state], key=self.Qtable[state].get)  # Selects the action with the highest Q-value from the current state
            next_states = [new_state for act, new_state in self.env.get_possible_acts(state) if act == action]  # Find possible next states for selected action from current state
            if not next_states:  # Breaks loop if no valid next states
                break
            if next_states[0] == state:  # Breaks loop if next state = current state
                break

            state = next_states[0]
            path.append(state)
        return path

    def is_optimal_path(self, path):
        return path[-1] == (x_T, y_T)

    def reset(self):
        self.Qtable.clear()
        self.model.clear()
        self.env.reset()



      
            


def find_start_of_optimal_solution(arr):
    if not arr:
        return -1  # Return -1 or handle the case where arr is empty

    min_value = min(arr)
    n = len(arr)

    for i in range(n):
        if arr[i] == min_value:
            if all(x == min_value for x in arr[i:]):
                return i
    return -1

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(400, 200)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setContentsMargins(20, 20, 20, 20)
        self.verticalLayout.setSpacing(20)
        self.verticalLayout.setAlignment(QtCore.Qt.AlignCenter)
        
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setText("Select the Size of the Grid (n) :")
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.verticalLayout.addWidget(self.label)

        self.comboBox = QtWidgets.QComboBox(self.centralwidget)
        self.comboBox.setObjectName("comboBox")
        for i in range(2, 11):
            self.comboBox.addItem(str(i))
        self.verticalLayout.addWidget(self.comboBox)

        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setObjectName("pushButton")
        self.pushButton.setText("Continue")
        self.pushButton.clicked.connect(self.open_grid_window)
        self.verticalLayout.addWidget(self.pushButton)

        MainWindow.setCentralWidget(self.centralwidget)

    def open_grid_window(self):
        n = int(self.comboBox.currentText())
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_GridWindow(n)
        self.ui.setupUi(self.window)
        self.window.show()
        MainWindow.close()

class Ui_GridWindow(object):
    def __init__(self, n):
        self.n = n
        self.target_set = False
        self.start_set = False
        self.block_mode = False
        self.grid_buttons = []

    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")

        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setContentsMargins(30, 30, 30, 30)
        self.verticalLayout.setSpacing(20)

        self.label_n = QtWidgets.QLabel(self.centralwidget)
        self.label_n.setText(f"Grid Size: {self.n} x {self.n}")
        self.label_n.setAlignment(QtCore.Qt.AlignLeft)
        self.verticalLayout.addWidget(self.label_n)

        self.gridLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.gridLayout = QtWidgets.QGridLayout(self.gridLayoutWidget)
        self.gridLayout.setContentsMargins(20, 20, 20, 20)
        self.gridLayout.setSpacing(20)
        self.gridLayout.setAlignment(QtCore.Qt.AlignCenter)
        self.gridLayoutWidget.setLayout(self.gridLayout)
        self.verticalLayout.addWidget(self.gridLayoutWidget)

        # Set the grid layout based on n
        for i in range(self.n):
            row_buttons = []
            for j in range(self.n):
                button = QtWidgets.QPushButton(self.gridLayoutWidget)
                button.setFixedSize(50, 50)
                button.clicked.connect(lambda ch, x=i, y=j: self.grid_button_clicked(x, y))
                self.gridLayout.addWidget(button, i, j)
                row_buttons.append(button)
            self.grid_buttons.append(row_buttons)

        self.continueButton = QtWidgets.QPushButton(self.centralwidget)
        self.continueButton.setText("Set Target")
        self.continueButton.clicked.connect(self.set_target_mode)
        self.verticalLayout.addWidget(self.continueButton)

        MainWindow.setCentralWidget(self.centralwidget)
        
        # Calculate the window size
        gridSize = self.gridLayout.sizeHint()
        windowWidth = gridSize.width() + 100  # Increase padding
        windowHeight = gridSize.height() + 100  # Increase padding
        MainWindow.setGeometry(100, 100, windowWidth, windowHeight)

    def set_target_mode(self):
        self.target_set = True
        self.continueButton.setText("Set Start")
        self.continueButton.clicked.disconnect()
        self.continueButton.clicked.connect(self.set_start_mode)

    def set_start_mode(self):
        self.start_set = True
        self.continueButton.setText("Set Blocked and Finalize")
        self.continueButton.clicked.disconnect()
        self.continueButton.clicked.connect(self.finalize_grid)

    def grid_button_clicked(self, x, y):
        button = self.grid_buttons[x][y]
        if button.text() != "" and button.text() != "Blocked":  # Allow action only if the cell is empty
            return
        if not self.target_set:
            self.clear_previous("Target")
            button.setStyleSheet("background-color: red;")
            button.setText("Target")
        elif not self.start_set:
            self.clear_previous("Start")
            button.setStyleSheet("background-color: green;")
            button.setText("Start")
        elif not self.block_mode:
            if button.text() == "Blocked":
                button.setStyleSheet("")
                button.setText("")
            else:
                button.setStyleSheet("background-color: grey;")
                button.setText("Blocked")

    def clear_previous(self, text):
        for row in self.grid_buttons:
            for button in row:
                if button.text() == text:
                    button.setStyleSheet("")
                    button.setText("")

    def finalize_grid(self):
        grid_representation = []
        for i in range(self.n):
            row = []
            for j in range(self.n):
                row.append(self.grid_buttons[i][j].text() if self.grid_buttons[i][j].text() else "Empty")
            grid_representation.append(row)
        
        print("Final Grid Representation:")
        for row in grid_representation:
            print(row)

        self.open_algorithm_selection_window(grid_representation)

    def open_algorithm_selection_window(self, grid_representation):
        self.algorithm_window = QtWidgets.QMainWindow()
        self.algorithm_ui = Ui_AlgorithmSelectionWindow(self.n, grid_representation, self.grid_buttons)
        self.algorithm_ui.setupUi(self.algorithm_window)
        self.algorithm_window.show()

class Ui_AlgorithmSelectionWindow(object):
    def __init__(self, n, grid_representation, grid_buttons):
        self.n = n
        self.grid_representation = grid_representation
        self.grid_buttons = grid_buttons

    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")

        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setContentsMargins(10, 10, 10, 10)
        self.verticalLayout.setSpacing(10)
        self.verticalLayout.setAlignment(QtCore.Qt.AlignCenter)

        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setText("Choose an Option")
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.verticalLayout.addWidget(self.label)

        self.button_qlearning = QtWidgets.QPushButton(self.centralwidget)
        self.button_qlearning.setText("Q-learning")
        self.button_qlearning.clicked.connect(lambda: self.show_final_grid("Q-learning"))
        self.verticalLayout.addWidget(self.button_qlearning)

        self.button_dyna_q = QtWidgets.QPushButton(self.centralwidget)
        self.button_dyna_q.setText("DynaQ")
        self.button_dyna_q.clicked.connect(lambda: self.show_final_grid("DynaQ"))
        self.verticalLayout.addWidget(self.button_dyna_q)

        self.button_dyna_q_plus = QtWidgets.QPushButton(self.centralwidget)
        self.button_dyna_q_plus.setText("DynaQ+")
        self.button_dyna_q_plus.clicked.connect(lambda: self.show_final_grid("DynaQ+"))
        self.verticalLayout.addWidget(self.button_dyna_q_plus)

#changes made for Prioritized Sweeping
        self.button_PrioritizedSweeping = QtWidgets.QPushButton(self.centralwidget)
        self.button_PrioritizedSweeping.setText("Prioritized-Sweeping")
        self.button_PrioritizedSweeping.clicked.connect(lambda: self.show_final_grid("Prioritized-Sweeping"))
        self.verticalLayout.addWidget(self.button_PrioritizedSweeping)

        self.button_compare = QtWidgets.QPushButton(self.centralwidget)
        self.button_compare.setText("Compare")
        self.button_compare.clicked.connect(lambda: self.show_final_grid("Compare"))
        self.verticalLayout.addWidget(self.button_compare)

        MainWindow.setCentralWidget(self.centralwidget)
        MainWindow.setGeometry(100, 100, 300, 250)

    def show_final_grid(self, algorithm):
        print(f"Selected Algorithm: {algorithm}")
        global start
        grid_representation = []
        blocked_tiles = []
        for i in range(self.n):
            row = []
            for j in range(self.n):
                if self.grid_buttons[i][j].text() == "Blocked":
                    blocked_tiles.append((i, j))
                row.append(self.grid_buttons[i][j].text() if self.grid_buttons[i][j].text() else "Empty")
            grid_representation.append(row)
        
        # print("Final Grid Representation:")
        # for row in grid_representation:
        #     print(row)
        
        maze_env = Grid(self.n, blocked_tiles)
        global x_T, y_T  # Declare global variables for the target position
        x_T, y_T = None, None
        for i in range(self.n):
            for j in range(self.n):
                if grid_representation[i][j] == "Start":
                    start = (i, j)
                elif grid_representation[i][j] == "Target":
                    x_T, y_T = i, j
        player = Player(maze_env)


        if (algorithm == "Q-learning"):
            player.train_Q()
            optimal_path = player.path(start)
            self.display_colored_grid(grid_representation, optimal_path)
            print("Optimal Path (Q-learning):", optimal_path)
            print("Number of steps taken by agent to reach target (Q-learning):", len(optimal_path) - 1)
            


        elif (algorithm == "DynaQ"):
            player.train_DynaQ()
            optimal_path = player.path(start)
            self.display_colored_grid(grid_representation, optimal_path)
            print("Optimal Path (Dyna-Q):", optimal_path)
            print("Number of steps taken by agent to reach target (Dyna-Q):", len(optimal_path) - 1)
            


        elif(algorithm == "DynaQ+"):
            player.train_DynaQPlus()
            optimal_path = player.path(start)
            self.display_colored_grid(grid_representation, optimal_path)
            print("Optimal Path (Dyna-Q+):", optimal_path)
            print("Number of steps taken by agent to reach target (Dyna-Q+):", len(optimal_path) - 1)

        elif(algorithm == "Prioritized-Sweeping"):
            player.train_PrioritizedSweeping()
            optimal_path = player.path(start)
            self.display_colored_grid(grid_representation, optimal_path)
            print("Optimal Path (PrioritizedSweeping):", optimal_path)
            print("Number of steps taken by agent to reach target (PrioritizedSweeping):", len(optimal_path) - 1)
            

        else:
            player1 = Player(maze_env)
            player2 = Player(maze_env)
            player3 = Player(maze_env)
            player4 = Player(maze_env)
            # player1.train_Q()
            # optimal_path1 = player1.path(start)
            # self.display_colored_grid(grid_representation, optimal_path1)
            # player2.train_DynaQ()
            # optimal_path2 = player2.path(start)
            # self.display_colored_grid(grid_representation, optimal_path2)
            # player3.train_DynaQPlus()
            # optimal_path3 = player3.path(start)
            # self.display_colored_grid(grid_representation, optimal_path3)
            # player.reset()
            global train_ep
            if(self.n == 2) : train_ep = 10
            elif(self.n == 3 ) : train_ep = 30
            elif(self.n == 4 ) : train_ep = 120
            elif(self.n == 5 ) : train_ep = 400
            elif(self.n == 6 ) : train_ep = 800
            elif(self.n == 7 ) : train_ep = 1100
            elif(self.n == 8 ) : train_ep = 1500
            elif(self.n == 9 ) : train_ep = 2000
            elif(self.n == 10 ) : train_ep = 2500
                
            player1.train_Q(train_ep)
            Q_steps_per_ep = player1.Q_steps_per_ep

            # player.reset()
            player2.train_DynaQ(train_ep)
            DynaQ_steps_per_ep = player2.dynaQ_steps_per_ep

            # player.reset()
            player3.train_DynaQPlus(train_ep)
            DynaQPlus_steps_per_ep = player3.dynaQPlus_steps_per_ep

            player4.train_PrioritizedSweeping(train_ep)
            PrioritizedSweeping_steps_per_ep = player4.PrioritizedSweeping_steps_per_ep

            # Plot the comparison graph

            fig, axs = plt.subplots(4, 1, figsize=(10, 15), sharex=True)
            
            # Define the number of episodes for x-ticks
            xticks = range(0, train_ep, int(train_ep/20))
            
            # Plot Q-learning
            axs[0].plot(Q_steps_per_ep, label='Q-learning', color='b')
            axs[0].set_ylabel('Number of Steps')
            axs[0].set_title('Q-learning')
            axs[0].legend()
            axs[0].set_xticks(xticks)
            
            # Plot Dyna-Q
            axs[1].plot(DynaQ_steps_per_ep, label='Dyna-Q', color='g')
            axs[1].set_ylabel('Number of Steps')
            axs[1].set_title('Dyna-Q')
            axs[1].legend()
            axs[1].set_xticks(xticks)
            
            # Plot Dyna-Q+
            axs[2].plot(DynaQPlus_steps_per_ep, label='Dyna-Q+', color='r')
            axs[2].set_ylabel('Number of Steps')
            axs[2].set_title('Dyna-Q+')
            axs[2].legend()
            axs[2].set_xticks(xticks)
            
            # Plot Prioritized Sweeping
            axs[3].plot(PrioritizedSweeping_steps_per_ep, label='Prioritized Sweeping', color='black')
            axs[3].set_xlabel('Episodes')
            axs[3].set_ylabel('Number of Steps')
            axs[3].set_title('Prioritized Sweeping')
            axs[3].legend()
            axs[3].set_xticks(xticks)
            
            plt.suptitle('Comparison of Q-learning, Dyna-Q, Dyna-Q+, and Prioritized Sweeping')
            plt.tight_layout(rect=[0, 0, 1, 0.96])
            plt.show()

            print("Number of episodes for Q-learning agent to find optimal path:",find_start_of_optimal_solution(player1.Q_steps_per_ep))
            print("Number of episodes for Dyna-Q agent to find optimal path:",find_start_of_optimal_solution(player2.dynaQ_steps_per_ep))
            print("Number of episodes for Dyna-Q-Plus agent to find optimal path:",find_start_of_optimal_solution(player3.dynaQPlus_steps_per_ep))
            print("Number of episodes for Prioritized Sweeping agent to find optimal path:",find_start_of_optimal_solution(player4.PrioritizedSweeping_steps_per_ep))


    def display_colored_grid(self, grid_representation, optimal_path):
        self.color_window = QtWidgets.QMainWindow()
        self.color_centralwidget = QtWidgets.QWidget(self.color_window)
        self.color_gridLayout = QtWidgets.QGridLayout(self.color_centralwidget)
        self.color_centralwidget.setLayout(self.color_gridLayout)

        color_mapping = {
            "Empty": "white",
            "Start": "green",
            "Blocked": "black",
            "Target": "red",
            "Path": "yellow"
        }

        for i in range(self.n):
            for j in range(self.n):
                tile_type = grid_representation[i][j]
                tile_color = color_mapping[tile_type]
                tile_label = QtWidgets.QLabel(self.color_centralwidget)
                tile_label.setStyleSheet(f"background-color: {tile_color}; border: 1px solid black;")
                tile_label.setFixedSize(50, 50)
                if tile_type != "Empty":
                    tile_label.setText(tile_type)
                    tile_label.setAlignment(QtCore.Qt.AlignCenter)
                self.color_gridLayout.addWidget(tile_label, i, j)

        # Highlight the optimal path
        for x, y in optimal_path[1:-1]:
            tile_label = QtWidgets.QLabel(self.color_centralwidget)
            tile_label.setStyleSheet("background-color: yellow; border: 1px solid black;")
            tile_label.setFixedSize(50, 50)
            tile_label.setText("Path")
            tile_label.setAlignment(QtCore.Qt.AlignCenter)
            self.color_gridLayout.addWidget(tile_label, x, y)

        self.color_window.setCentralWidget(self.color_centralwidget)
        self.color_window.setGeometry(100, 100, self.n * 50 + 40, self.n * 50 + 40)
        self.color_window.show()
        

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

Final Grid Representation:
['Start', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Blocked', 'Empty']
['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Blocked', 'Empty']
['Empty', 'Empty', 'Empty', 'Empty', 'Blocked', 'Empty', 'Empty', 'Empty']
['Blocked', 'Blocked', 'Blocked', 'Blocked', 'Empty', 'Empty', 'Blocked', 'Empty']
['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Blocked', 'Empty']
['Empty', 'Blocked', 'Empty', 'Blocked', 'Blocked', 'Blocked', 'Blocked', 'Blocked']
['Empty', 'Empty', 'Empty', 'Blocked', 'Empty', 'Empty', 'Empty', 'Empty']
['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Target']
Selected Algorithm: Q-learning
Optimal Path (Q-learning): [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (3, 4), (4, 4), (4, 3), (4, 2), (5, 2), (6, 2), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
Number of steps taken by agent to reach target (Q-learning): 20
Selected Algorithm: DynaQ+
Optimal Path (Dyna-Q+): [(0, 0), (0, 1